In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=de6c5f695058cbf65e41276608a31a027fb57332f69efb58a57642a01b9983db
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Loan approval").getOrCreate()

In [ ]:
sc = spark.sparkContext

In [ ]:
data = spark.read.option('header',True).csv('/content/loan_approval_dataset.csv')

In [ ]:
data.show()

+-------+-----------------+-------------+--------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+
|loan_id| no_of_dependents|    education| self_employed| income_annum| loan_amount| loan_term| cibil_score| residential_assets_value| commercial_assets_value| luxury_assets_value| bank_asset_value| loan_status|
+-------+-----------------+-------------+--------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+
|      1|                2|     Graduate|            No|      9600000|    29900000|        12|         778|                  2400000|                17600000|            22700000|          8000000|    Approved|
|      2|                0| Not Graduate|           Yes|      4100000|    12200000|         8|         417|                  2700000|                 220000

In [ ]:
data.printSchema()

root
 |-- loan_id: string (nullable = true)
 |--  no_of_dependents: string (nullable = true)
 |--  education: string (nullable = true)
 |--  self_employed: string (nullable = true)
 |--  income_annum: string (nullable = true)
 |--  loan_amount: string (nullable = true)
 |--  loan_term: string (nullable = true)
 |--  cibil_score: string (nullable = true)
 |--  residential_assets_value: string (nullable = true)
 |--  commercial_assets_value: string (nullable = true)
 |--  luxury_assets_value: string (nullable = true)
 |--  bank_asset_value: string (nullable = true)
 |--  loan_status: string (nullable = true)



In [ ]:
data.count()

4269

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [ ]:
#CATEGORICAL COLUMNS : EDUCATION, SELF EMPLOYED AND APPROVAL:::

In [ ]:
indexer = StringIndexer(inputCol=' self_employed', outputCol='self_employed_numeric').fit(data)
data = indexer.transform(data)

In [ ]:
data.show()

+-------+-----------------+-------------+--------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+---------------------+
|loan_id| no_of_dependents|    education| self_employed| income_annum| loan_amount| loan_term| cibil_score| residential_assets_value| commercial_assets_value| luxury_assets_value| bank_asset_value| loan_status|self_employed_numeric|
+-------+-----------------+-------------+--------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+---------------------+
|      1|                2|     Graduate|            No|      9600000|    29900000|        12|         778|                  2400000|                17600000|            22700000|          8000000|    Approved|                  1.0|
|      2|                0| Not Graduate|           Yes|      410000

In [ ]:
data  = data.drop(' self_employed')
data.show()

+-------+-----------------+-------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+---------------------+
|loan_id| no_of_dependents|    education| income_annum| loan_amount| loan_term| cibil_score| residential_assets_value| commercial_assets_value| luxury_assets_value| bank_asset_value| loan_status|self_employed_numeric|
+-------+-----------------+-------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+---------------------+
|      1|                2|     Graduate|      9600000|    29900000|        12|         778|                  2400000|                17600000|            22700000|          8000000|    Approved|                  1.0|
|      2|                0| Not Graduate|      4100000|    12200000|         8|         417|                  2700000|          

In [ ]:
indexer = StringIndexer(inputCol = ' education',outputCol = 'education_numeric').fit(data)
data = indexer.transform(data)

In [ ]:
data.show()

+-------+-----------------+-------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+---------------------+-----------------+
|loan_id| no_of_dependents|    education| income_annum| loan_amount| loan_term| cibil_score| residential_assets_value| commercial_assets_value| luxury_assets_value| bank_asset_value| loan_status|self_employed_numeric|education_numeric|
+-------+-----------------+-------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+---------------------+-----------------+
|      1|                2|     Graduate|      9600000|    29900000|        12|         778|                  2400000|                17600000|            22700000|          8000000|    Approved|                  1.0|              0.0|
|      2|                0| Not Graduate|      4100000| 

In [ ]:
data = data.drop(' education')
data.show()

+-------+-----------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+---------------------+-----------------+
|loan_id| no_of_dependents| income_annum| loan_amount| loan_term| cibil_score| residential_assets_value| commercial_assets_value| luxury_assets_value| bank_asset_value| loan_status|self_employed_numeric|education_numeric|
+-------+-----------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+---------------------+-----------------+
|      1|                2|      9600000|    29900000|        12|         778|                  2400000|                17600000|            22700000|          8000000|    Approved|                  1.0|              0.0|
|      2|                0|      4100000|    12200000|         8|         417|                  2700000|        

In [ ]:
indexer = StringIndexer(inputCol = ' loan_status',outputCol = 'loan_status').fit(data)
data = indexer.transform(data)

In [ ]:
data.show()

+-------+-----------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+---------------------+-----------------+-----------+
|loan_id| no_of_dependents| income_annum| loan_amount| loan_term| cibil_score| residential_assets_value| commercial_assets_value| luxury_assets_value| bank_asset_value| loan_status|self_employed_numeric|education_numeric|loan_status|
+-------+-----------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+------------+---------------------+-----------------+-----------+
|      1|                2|      9600000|    29900000|        12|         778|                  2400000|                17600000|            22700000|          8000000|    Approved|                  1.0|              0.0|        0.0|
|      2|                0|      4100000|    12200000|         8

In [ ]:
data = data.drop(' loan_status')
data.show()

+-------+-----------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+---------------------+-----------------+-----------+
|loan_id| no_of_dependents| income_annum| loan_amount| loan_term| cibil_score| residential_assets_value| commercial_assets_value| luxury_assets_value| bank_asset_value|self_employed_numeric|education_numeric|loan_status|
+-------+-----------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+---------------------+-----------------+-----------+
|      1|                2|      9600000|    29900000|        12|         778|                  2400000|                17600000|            22700000|          8000000|                  1.0|              0.0|        0.0|
|      2|                0|      4100000|    12200000|         8|         417|                  2700000|            

In [ ]:
data.printSchema()

root
 |-- loan_id: string (nullable = true)
 |--  no_of_dependents: string (nullable = true)
 |--  income_annum: string (nullable = true)
 |--  loan_amount: string (nullable = true)
 |--  loan_term: string (nullable = true)
 |--  cibil_score: string (nullable = true)
 |--  residential_assets_value: string (nullable = true)
 |--  commercial_assets_value: string (nullable = true)
 |--  luxury_assets_value: string (nullable = true)
 |--  bank_asset_value: string (nullable = true)
 |-- self_employed_numeric: double (nullable = false)
 |-- education_numeric: double (nullable = false)
 |-- loan_status: double (nullable = false)



In [ ]:
inputcolumns = [' no_of_dependents',' income_annum',' loan_amount',' loan_term',' cibil_score',' residential_assets_value',' commercial_assets_value',' luxury_assets_value',' bank_asset_value']

In [ ]:
for i in inputcolumns:
  data = data.withColumn(i,data[i].cast(IntegerType()))

In [ ]:
data.printSchema()

root
 |-- loan_id: string (nullable = true)
 |--  no_of_dependents: integer (nullable = true)
 |--  income_annum: integer (nullable = true)
 |--  loan_amount: integer (nullable = true)
 |--  loan_term: integer (nullable = true)
 |--  cibil_score: integer (nullable = true)
 |--  residential_assets_value: integer (nullable = true)
 |--  commercial_assets_value: integer (nullable = true)
 |--  luxury_assets_value: integer (nullable = true)
 |--  bank_asset_value: integer (nullable = true)
 |-- self_employed_numeric: double (nullable = false)
 |-- education_numeric: double (nullable = false)
 |-- loan_status: double (nullable = false)



In [ ]:
inputcolumn = [' no_of_dependents',' income_annum',' loan_amount',' loan_term',' cibil_score',' residential_assets_value',' commercial_assets_value',' luxury_assets_value',' bank_asset_value','self_employed_numeric','education_numeric']

In [ ]:
d = VectorAssembler(inputCols = inputcolumn, outputCol = 'attributes')
data = d.transform(data)

In [ ]:
data.show()

+-------+-----------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+---------------------+-----------------+-----------+--------------------+
|loan_id| no_of_dependents| income_annum| loan_amount| loan_term| cibil_score| residential_assets_value| commercial_assets_value| luxury_assets_value| bank_asset_value|self_employed_numeric|education_numeric|loan_status|          attributes|
+-------+-----------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+---------------------+-----------------+-----------+--------------------+
|      1|                2|      9600000|    29900000|        12|         778|                  2400000|                17600000|            22700000|          8000000|                  1.0|              0.0|        0.0|[2.0,9600000.0,2....|
|      2|                0|     

In [ ]:
train, test = data.randomSplit([0.8,0.2], seed=11)

In [ ]:
train.count()

3437

In [ ]:
test.count()

832

In [ ]:
from pyspark.ml.classification import LogisticRegression

log = LogisticRegression(featuresCol='attributes',
                             labelCol='loan_status')

In [ ]:
pipe = Pipeline(stages = [log])

In [ ]:
fit_model = pipe.fit(train)

In [ ]:
results = fit_model.transform(test)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
res = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='loan_status')
efficiency = res.evaluate(results)

In [ ]:
print("Accuracy of the model is: ",efficiency*100,"%")

Accuracy of the model is:  89.37228915662652 %


In [ ]:
results.show()

+-------+-----------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+---------------------+-----------------+-----------+--------------------+--------------------+--------------------+----------+
|loan_id| no_of_dependents| income_annum| loan_amount| loan_term| cibil_score| residential_assets_value| commercial_assets_value| luxury_assets_value| bank_asset_value|self_employed_numeric|education_numeric|loan_status|          attributes|       rawPrediction|         probability|prediction|
+-------+-----------------+-------------+------------+----------+------------+-------------------------+------------------------+--------------------+-----------------+---------------------+-----------------+-----------+--------------------+--------------------+--------------------+----------+
|   1002|                1|      4900000|    17700000|         8|         722|                  9200000|           